# Dependencies

In [ ]:
#@title TF Quantization
!pip install -q tensorflow-model-optimization

In [ ]:
#@title PESC - Spearmint

# MongoDb
!apt install -y mongodb
!pip2.7 install pymongo
!mkdir mongodb
!mongod --fork --logpath mongolog --dbpath mongodb

# Spearmint
!pip2.7 install numpy==1.10
!pip2.7 install scipy==0.18.1
!pip2.7 install weave
!git clone https://github.com/HIPS/Spearmint.git
!cd Spearmint && git checkout PESC
!pip2.7 install -e Spearmint

In [ ]:
#@title CMA-ES - Pycma

!pip install cma

In [ ]:
#@title cEI - GPflow

!git clone https://github.com/GPflow/GPflow.git
!cd GPflow && python setup.py develop

!pip install pip==18.1
!pip install git+https://github.com/GPflow/GPflowOpt.git --process-dependency-links

In [ ]:
#@title COBYLA - scikit-optimize
!pip install scikit-optimize

# Definition

In [ ]:
#@title Base import and seed

import os
import math
import random
import numpy as np
import tensorflow as tf

import sys
if not 'emllib' in sys.path: sys.path.insert(1, 'emllib')

import pickle

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

In [ ]:
#@title Quantization definitions

import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.quantization.keras.default_8bit import default_8bit_quantize_scheme, default_8bit_quantize_registry, default_8bit_quantizers
from tensorflow_model_optimization.quantization.keras.quantizers import LastValueQuantizer, MovingAverageQuantizer
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer as ql
from tensorflow_model_optimization.quantization.keras import quantize_annotate_model, QuantizeConfig
import tensorflow as tf

class ConvWeightsQuantizer(LastValueQuantizer):
  """Quantizer for handling weights in Conv2D/DepthwiseConv2D layers."""
  def __init__(self, bits):
        super(ConvWeightsQuantizer, self).__init__(
            num_bits=bits, per_axis=True, symmetric=True, narrow_range=True)
  def build(self, tensor_shape, name, layer):
    min_weight = layer.add_weight(
        name + '_min',
        shape=(tensor_shape[-1],),
        initializer=tf.keras.initializers.Constant(-6.0),
        trainable=False)
    max_weight = layer.add_weight(
        name + '_max',
        shape=(tensor_shape[-1],),
        initializer=tf.keras.initializers.Constant(6.0),
        trainable=False)
    return {'min_var': min_weight, 'max_var': max_weight}

class QConf(default_8bit_quantize_registry.Default8BitQuantizeConfig):
    def __init__(self, bits, conv, *args, **kwargs):
        super(QConf, self).__init__(*args, **kwargs)
        self.bits = bits
        if conv:
            self.weight_quantizer = ConvWeightsQuantizer(bits)
        else:
            self.weight_quantizer = LastValueQuantizer(
                num_bits=self.bits, per_axis=False, symmetric=True, narrow_range=True)
        self.activation_quantizer = MovingAverageQuantizer(
            num_bits=self.bits, per_axis=False, symmetric=False, narrow_range=False)

class QAct(default_8bit_quantize_registry.Default8BitActivationQuantizeConfig):
    def __init__(self, bits, *args, **kwargs):
        super(QAct, self).__init__(*args, **kwargs)
        self.bits = bits

    def get_output_quantizers(self, layer):
        self._assert_activation_layer(layer)
        if not hasattr(layer.activation, '__name__'):
            raise ValueError('Activation {} not supported by '
                            'Default8BitActivationQuantizeConfig.'.format(
                                layer.activation))
        if layer.activation.__name__ in ['relu', 'swish']:
            return [MovingAverageQuantizer(
            num_bits=self.bits, per_axis=False, symmetric=False, narrow_range=False)]
        elif layer.activation.__name__ in ['linear', 'softmax', 'sigmoid', 'tanh']:
            return []
        raise ValueError('Activation {} not supported by '
                        'Default8BitActivationQuantizeConfig.'.format(
                            layer.activation))

class QReg(default_8bit_quantize_registry.Default8BitQuantizeRegistry):
    def __init__(self, bitlist, *args, **kwargs):
        super(QReg, self).__init__(*args, **kwargs)
        self.bitlist = bitlist
        self.counter = -1

    def get_quantize_config(self, layer):
        self.counter += 1
        quantize_info = self._get_quantize_info(layer.__class__)
        if layer.name.startswith('activation'):
            return QAct(self.bitlist[self.counter])
        return QConf(self.bitlist[self.counter], 
                     layer.name.startswith('conv'),  # enable ConvWeightsQuantizer
                     quantize_info.weight_attrs,
                     quantize_info.activation_attrs,
                     quantize_info.quantize_output)

class QScheme(default_8bit_quantize_scheme.Default8BitQuantizeScheme):
    def __init__(self, bitlist, *args, **kwargs):
        super(QScheme, self).__init__(*args, **kwargs)
        self.bitlist = bitlist

    def get_quantize_registry(self):
        return QReg(self.bitlist)

In [ ]:
#@title Pull pretrained model

cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
dataset_mean = train_images.mean(axis=(0,1,2))
dataset_std = train_images.std(axis=(0,1,2))
train_images = (train_images - dataset_mean) / dataset_std
test_images = (test_images - dataset_mean) / dataset_std

!mkdir pretrained_resnet18
!mkdir pretrained_resnet18/variables
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/d9d4d8f866df84014e528bb3c5617816 -O  pretrained_resnet18/variables/variables.data-00000-of-00001
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/4901af0e55327757ca7d7380b353279f -O  pretrained_resnet18/variables/variables.index
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/0a1a30ebb8498c7adaab17365283b563 -O  pretrained_resnet18/keras_metadata.pb
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/7c8a4682f521bac78f8a89b70342675b -O  pretrained_resnet18/saved_model.pb

In [ ]:
#@title Objective & Constraint

y_samples = []

def tpc(x):
    print("Query for x=", x)
    model = tf.keras.models.load_model("pretrained_resnet18")
    model = quantize_annotate_model(model)
    q_aware_model = tfmot.quantization.keras.quantize_apply(model, QScheme(x))
    q_aware_model.compile(optimizer="adam",
                    loss='sparse_categorical_crossentropy', metrics=["accuracy"])

    q_aware_model.fit(train_images, train_labels,
                    batch_size=512, epochs=5, validation_split=0.1,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                        patience=5, restore_best_weights=True)])
    
    q_aware_model_loss, q_aware_model_accuracy = q_aware_model.evaluate(
        test_images, test_labels, verbose=0)
    
    y_samples.append(-q_aware_model_accuracy)

    wandb.log({"x": x, "y": -q_aware_model_accuracy, "y_min": min(y_samples),
               "cst_violation": cst(x)})
    return -q_aware_model_accuracy

def cst(x):
    return len(x)*4 - sum(x)

In [ ]:
#@title WandB

# set if you plan to log on wandb
ENABLE_WANDB = True                            #@param {type:"boolean"}        

if ENABLE_WANDB and "wandb" not in sys.modules:
    !pip install wandb > /dev/null
    !wandb login
    import wandb

def init_wandb(experiment_name, run_id=None):
    if run_id is not None:
        wandb.init(project='eml', id=run_id, resume='allow')
    else:
        wandb.init(project='eml', name=experiment_name)

# Run

In [ ]:
#@title PESC - Spearmint
!cd Spearmint/spearmint && python2 main.py ../examples/constrained

In [ ]:
#@title CMA-ES - Pycma

import cma

es = cma.CMAEvolutionStrategy(8*[0], 0.5)
es.optimize(cma.ff.rosen, iterations=20)

In [ ]:
#@title cEI - GPFlowOpt

import gpflow
import gpflowopt
import numpy as np

# Objective & constraint
def townsend(X):
    return -(np.cos((X[:,0]-0.1)*X[:,1])**2 + X[:,0] * np.sin(3*X[:,0]+X[:,1]))[:,None]

def constraint(X):
    return -(-np.cos(1.5*X[:,0]+np.pi)*np.cos(1.5*X[:,1])+np.sin(1.5*X[:,0]+np.pi)*np.sin(1.5*X[:,1]))[:,None]

# Setup input domain
domain = gpflowopt.domain.ContinuousParameter('x1', -2.25, 2.5) + \
         gpflowopt.domain.ContinuousParameter('x2', -2.5, 1.75)

# Plot
def plotfx():
    X = gpflowopt.design.FactorialDesign(101, domain).generate()
    Zo = townsend(X)
    Zc = constraint(X)
    mask = Zc>=0
    Zc[mask] = np.nan
    Zc[np.logical_not(mask)] = 1
    Z = Zo * Zc
    shape = (101, 101)

    f, axes = plt.subplots(1, 1, figsize=(7, 5))
    axes.contourf(X[:,0].reshape(shape), X[:,1].reshape(shape), Z.reshape(shape))
    axes.set_xlabel('x1')
    axes.set_ylabel('x2')
    axes.set_xlim([domain.lower[0], domain.upper[0]])
    axes.set_ylim([domain.lower[1], domain.upper[1]])
    return axes


In [ ]:
#@title Cobyla - Scipy

from scipy.optimize import fmin_cobyla

init_wandb("COBYLA_tpc")
set_seed()

fmin_cobyla(tpc, [4]*41, [cst], rhoend=1e-7, maxfun=200)